#### This file use NBeat from 
https://pypi.org/project/nbeats-forecast/

#### We need to check it also
https://pypi.org/project/nbeats-pytorch/



In [32]:
import pandas as pd
from datetime import timedelta
from nbeats_forecast import NBeats
import os
import sys
import warnings
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib widget
warnings.filterwarnings("ignore")

In [33]:
sys.path.append('src')

In [34]:
from consumer import DataLoader

In [35]:
dl = DataLoader()
state ='KA'
ind_data = dl.get_all_timeseries()[state]
ind_data['delta.TPR'] = ind_data['delta.confirmed']/ind_data['delta.tested']
ind_data

,date,population,delta.confirmed,delta7.confirmed,total.confirmed,delta.deceased,delta7.deceased,total.deceased,delta.recovered,delta7.recovered,...,delta.other,delta7.other,total.other,delta.vaccinated1,delta7.vaccinated1,total.vaccinated1,delta.vaccinated2,delta7.vaccinated2,total.vaccinated2,delta.TPR
0,2020-03-09,65798000,1.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-10,65798000,3.0,4,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-11,65798000,NaN,4,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-12,65798000,1.0,5,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-13,65798000,1.0,6,6,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2021-07-20,65798000,1464.0,12105,2886702,29.0,282.0,36226.0,2706.0,19801.0,...,NaN,NaN,23.0,116504.0,939985.0,22305056.0,66803.0,628426.0,5475128.0,0.012904
499,2021-07-21,65798000,1639.0,11754,2888341,36.0,273.0,36262.0,2214.0,19478.0,...,NaN,NaN,23.0,94617.0,867874.0,22399673.0,82866.0,604460.0,5557994.0,0.010732
500,2021-07-22,65798000,1653.0,11430,2889994,31.0,256.0,36293.0,2572.0,18862.0,...,NaN,NaN,23.0,171326.0,960963.0,22570999.0,119700.0,673122.0,5677694.0,0.011778
501,2021-07-23,65798000,1705.0,11329,2891699,30.0,244.0,36323.0,2243.0,18357.0,...,NaN,NaN,23.0,185319.0,1014338.0,22756318.0,125700.0,693994.0,5803394.0,0.013594


In [36]:
ind_data =ind_data.fillna(0)

### Choose the compartment in string variable var

In [37]:
var = 'delta.confirmed'

In [38]:
df = ind_data[['date', var]]

df[[var]].plot(legend=True, figsize=(10, 6), title= var+ ' '+ state)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'delta.confirmed KA'}>

In [39]:
df_date = df['date']
data = df[var].to_numpy()
data = data.reshape(data.shape[0],1).astype('float64')

In [40]:
NBeats?

Init signature:
NBeats(
    period_to_forecast,
    data=None,
    backcast_length=None,
    save_checkpoint=False,
    path='',
    checkpoint_file_name='nbeats-training-checkpoint.th',
    mode='cpu',
    batch_size=None,
    thetas_dims=[7, 8],
    nb_blocks_per_stack=3,
    share_weights_in_stack=False,
    train_percent=0.8,
    hidden_layer_units=128,
    stack=None,
)
Docstring:      <no docstring>
File:           ~/.condaroot/envs/ml/lib/python3.8/site-packages/nbeats_forecast/nbeats_forecast.py
Type:           type
Subclasses:     


In [42]:
period_to_forecast = 15
model = NBeats(data=data, period_to_forecast=period_to_forecast, backcast_length=15)
model.fit(epoch=50)

TypeError: __init__() got an unexpected keyword argument 'thetas_dims'

In [50]:
period_backcast = 3*period_to_forecast
backcast_dates = df_date[-period_backcast:]

In [51]:
backcast_data = data[-period_backcast:]
forecast = model.predict(backcast_data)

In [52]:
df = pd.DataFrame(columns=['backdates'])
df['backdates'] = backcast_dates
df['futuredate'] =df['backdates'] + timedelta(period_to_forecast)
df['history'] = backcast_data
df['prediction'] =pd.DataFrame(columns=['prediction'])
df['prediction'][-period_to_forecast:]=forecast.flatten()

In [53]:
df_history = df[['backdates', 'history']].head(period_backcast)
df_history = df_history.set_index('backdates')
df_future = df[['futuredate', 'prediction']].tail(period_to_forecast)
df_future= df_future.set_index('futuredate')

In [54]:
ax = df_history.plot(legend=True, figsize=(10, 6), title=state + ' '+ var, marker='*')
df_future.plot(ax=ax, legend=True, figsize=(10, 6), title=state + ' ' +var, marker='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'KA delta.confirmed'}, xlabel='futuredate'>

,prediction
futuredate,
2021-07-24,1566.08
2021-07-25,1675.63
2021-07-26,1361.63
2021-07-27,1377.02
2021-07-28,1290
2021-07-29,1391.73
2021-07-30,1489.06
2021-07-31,1217.46
2021-08-01,1464.26
